In [2]:
from abc_utils import *
import pandas as pd
from hmmlearn import hmm
import numpy as np

In [3]:
train_set, train_lengths, val_set, _ = load_datasets()

In [4]:
def load_song(index):
    start_pos = train_lengths.iloc[:index].sum().item()
    return train_set.iloc[start_pos : start_pos + train_lengths.iloc[index].item()]

In [5]:
def load_songs(indices):
    end_positions = np.cumsum(train_lengths)
    positions = np.insert(end_positions, 0, np.array([0]))
    songs = []
    for i in indices:
        song = train_set.iloc[positions[i] : positions[i+1]]
        songs.append(song)
    # return songs
    return pd.concat(songs)

song_indices = list(range(10))
songs = load_songs(song_indices)

In [6]:
states, observations = dataframe_to_states(
    songs, 
    2, 
    1
)

Processing states: 100%|██████████| 944/944 [00:00<00:00, 12123.34it/s]


In [7]:
transition_matrix, emission_probs, unique_states, unique_obs, states_to_index, observation_to_index = states_to_transition(states, observations)

In [8]:
observation_indices = observations.astype(int) 

In [9]:
train_lengths.min()

lengths    7
dtype: int64

In [10]:
model = hmm.CategoricalHMM(
    n_components=transition_matrix.shape[0],
    init_params=''
)

# model.fit(observation_indices.reshape(-1, 1))

In [11]:
model.transmat_ = transition_matrix.T
model.emissionprob_ = emission_probs.T
start_probs = np.ones(transition_matrix.shape[0]) / transition_matrix.shape[0]
model.startprob_ = start_probs

Other tools:
- dynamax
  - runs on JAX
  - might have a more flexible structure
  - 

In [13]:
observation_indices

array([82, 80, 79, 77, 79, 75, 75, 74, 75, 77, 70, 74, 75, 77, 75, 77, 79,
       77, 79, 82, 80, 79, 77, 79, 75, 75, 74, 75, 77, 70, 74, 77, 75, 77,
       79, 80, 82, 87, 86, 87, 82, 82, 80, 79, 77, 75, 74, 72, 70, 72, 74,
       75, 77, 79, 82, 87, 86, 87, 82, 82, 80, 79, 77, 75, 74, 77, 75, 77,
       79, 80, 75, 77, 79, 80, 77, 79, 80, 82, 79, 80, 82, 84, 80, 82, 87,
       86, 87, 82, 79, 77, 79, 80, 77, 79, 80, 82, 79, 80, 79, 75, 74, 75,
       77, 74, 67, 60, 62, 65, 67, 69, 67, 65, 74, 72, 60, 62, 65, 67, 69,
       67, 65, 62, 60, 60, 62, 65, 67, 69, 67, 65, 74, 72, 72, 77, 74, 72,
       77, 74, 72, 69, 67, 65, 72, 77, 72, 74, 72, 69, 72, 81, 79, 77, 72,
       77, 72, 74, 72, 69, 74, 72, 69, 79, 72, 77, 72, 74, 72, 69, 72, 77,
       81, 77, 79, 77, 74, 77, 72, 77, 72, 74, 72, 77, 74, 72, 69, 67, 72,
       74, 72, 74, 77, 79, 81, 79, 77, 74, 77, 81, 79, 81, 84, 72, 74, 72,
       74, 77, 79, 81, 79, 77, 74, 70, 72, 72, 84, 84, 84, 86, 84, 82, 81,
       79, 77, 74, 77, 81

In [12]:
likelihood, pred_states = model.decode(observation_indices.reshape(-1, 1))
pred_states

IndexError: index 82 is out of bounds for axis 1 with size 24